# Get klines on Binance

In [11]:
import numpy as np
import pandas as pd
import requests

In [12]:
symbol = "BTCUSDT"
interval = "1m"

In [13]:
API_URL = 'https://api.binance.com/api/v3'

def get_historical(symbol, interval):
    '''
    Fetch historical prices from binance (klines)
    Prepare it and return it as a fresh new DataFrame

    Parameters:
    ===========
    symbol: str
        trading pair symbol (uppercase)
    interval: str
        tick interval: 1m, 3m, 5m, 15m, 30m, 1h, 2h, 4h, 6h, 8h, 12h, 1d, 3d, 1w, 1M

    Returns:

    '''

    try:
        url = f"{API_URL}/klines?symbol={symbol.upper()}&interval={interval}&limit=1000"
        data = requests.get(url).json()
    except requests.exceptions.RequestException as e:
        raise SystemExit(e)

    raws = []
    for tick in data:
        time = pd.to_datetime(tick[0], unit='ms')
        close = float(tick[4])
        raws.append([time, close])

    df = pd.DataFrame(raws, columns=['Date', 'price'])
    df.set_index('Date', inplace=True)
    df['return'] = np.log(df['price'] / df['price'].shift(1))
    return df.dropna()

In [14]:
df = get_historical(symbol, interval)

In [15]:
df.to_csv(f"{symbol}-{interval}.csv")